# 04f - Vertex AI > Training > Custom Jobs - Forecasting with Prophet using Custom Container


### Prerequisites:
-  04 - Time Series Forecasting - Data Review in BigQuery

### Overview:
-  

### Resources:
-  

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/04f_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/04f_console.png">

---
## Setup

inputs:

In [ ]:
PROJECT_ID='statmike-mlops'
REGION = 'us-central1'
DATANAME = 'citibikes'
NOTEBOOK = '04f'

packages:

clients:

parameters:

environment: